# 1) Chicago Car Crash Project

### Introduction

### Business Problem Statement

### Objectives

### Experimental Design Taken

### 2) Load the Dataset

In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [80]:
#load the dataset
crashes = pd.read_csv(r"C:\Users\wanji\OneDrive\Desktop\Traffic_Crashes_-_Crashes.csv", low_memory= False)
people = pd.read_csv(r"C:\Users\wanji\OneDrive\Desktop\Traffic_Crashes_-_People.csv",low_memory = False)
vehicles = pd.read_csv(r"C:\Users\wanji\OneDrive\Desktop\Traffic_Crashes_-_Vehicles.csv", low_memory = False)

In [81]:
# Merge the datsets
merged_data = pd.merge(crashes, vehicles, on = 'CRASH_RECORD_ID', how = 'inner')
merged_data = pd.merge(merged_data, people, on = 'CRASH_RECORD_ID',how = 'inner')
merged_data.head()

,CRASH_RECORD_ID,RD_NO_x,CRASH_DATE_EST_I,CRASH_DATE_x,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,...,EMS_RUN_NO,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
0,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,JC199149,NaN,03/25/2019 02:43:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,JC199149,NaN,03/25/2019 02:43:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,...,NaN,IMPROPER TURN,UNKNOWN,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
2,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,JC199149,NaN,03/25/2019 02:43:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,...,NaN,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
3,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,JC199149,NaN,03/25/2019 02:43:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,JC199149,NaN,03/25/2019 02:43:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,...,NaN,IMPROPER TURN,UNKNOWN,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN


### 2.1) Providing the description of the dataset: Shape, Statistics and more additional info about the merged datasets

In [82]:
#Function to give the shape/general description of the dataset
def describe_merged_dataset(merged_data):
    # Shape of the dataset
    shape = merged_data.shape
    print("Shape of the dataset:", shape)

    statistics = merged_data.describe()
    print("\nBasic Statistics:")
    print(statistics)
    
    # Additional information
    print("\nAdditional Information:")
    merged_data.info()

# In this case for the merged datasets we are provided with the following information
describe_merged_dataset(merged_data)

Shape of the dataset: (3324249, 149)

Basic Statistics:
       POSTED_SPEED_LIMIT      LANE_CNT     STREET_NO  BEAT_OF_OCCURRENCE  \
count        3.324249e+06  9.306350e+05  3.324249e+06        3.324225e+06   
mean         2.892263e+01  9.342349e+00  3.638574e+03        1.232284e+03   
std          5.716456e+00  2.109722e+03  2.870519e+03        7.019735e+02   
min          0.000000e+00  0.000000e+00  0.000000e+00        1.110000e+02   
25%          3.000000e+01  2.000000e+00  1.201000e+03        7.130000e+02   
50%          3.000000e+01  2.000000e+00  3.160000e+03        1.134000e+03   
75%          3.000000e+01  4.000000e+00  5.529000e+03        1.814000e+03   
max          9.900000e+01  1.191625e+06  4.511000e+05        6.100000e+03   

          NUM_UNITS  INJURIES_TOTAL  INJURIES_FATAL  INJURIES_INCAPACITATING  \
count  3.324249e+06    3.324249e+06    3.324249e+06             3.324249e+06   
mean   2.198258e+00    3.069602e-01    1.938784e-03             3.134904e-02   
std    6.5

In [83]:
# Identifying the columns and the data types in the merged datasets
def describe_columns(merged_data):
    # Columns and their data types
    columns = merged_data.columns
    data_types = merged_data.dtypes
    
    # Create a DataFrame to store the information
    column_info = pd.DataFrame({'Column Name': columns, 'Data Type': data_types})
    
    # Print the column information
    print(column_info)

describe_columns(merged_data)

                             Column Name Data Type
CRASH_RECORD_ID          CRASH_RECORD_ID    object
RD_NO_x                          RD_NO_x    object
CRASH_DATE_EST_I        CRASH_DATE_EST_I    object
CRASH_DATE_x                CRASH_DATE_x    object
POSTED_SPEED_LIMIT    POSTED_SPEED_LIMIT     int64
...                                  ...       ...
PEDPEDAL_VISIBILITY  PEDPEDAL_VISIBILITY    object
PEDPEDAL_LOCATION      PEDPEDAL_LOCATION    object
BAC_RESULT                    BAC_RESULT    object
BAC_RESULT VALUE        BAC_RESULT VALUE   float64
CELL_PHONE_USE            CELL_PHONE_USE    object

[149 rows x 2 columns]


### 2.1.1) Data Cleaning

#### Calculating for the missing values 
- Show the number of missing values 
- The percentage of missing values for each column in the dataset.

In [84]:
def calculate_missing_values(merged_data):
    # Count the missing values in each column
    missing_values = merged_data.isnull().sum()

    # Calculate the percentage of missing values for each column
    missing_percentage = (missing_values / len(merged_data)) * 100

    # Create a DataFrame to store the missing values and percentages
    missing_data = pd.DataFrame({'Column': missing_values.index, 'Missing Values': missing_values.values, 'Missing Percentage': missing_percentage.values})

    return missing_data


# Calculate the missing values
missing_data = calculate_missing_values(merged_data)

# Print the missing data
missing_data.head()

,Column,Missing Values,Missing Percentage
0,CRASH_RECORD_ID,0,0.000000
1,RD_NO_x,22061,0.663639
2,CRASH_DATE_EST_I,3136129,94.340977
3,CRASH_DATE_x,0,0.000000
4,POSTED_SPEED_LIMIT,0,0.000000


#### Extracting the missing values at 95% which is greater than the threshold

In [85]:
def extract_missing_values(dataset, threshold, columns_to_keep):
    # Calculate the missing values and percentages
    missing_data = calculate_missing_values(dataset)  # Replace merged_data with dataset

    # Extract the columns with missing values exceeding the threshold
    missing_above_threshold = missing_data[missing_data['Missing Percentage'] > threshold]

    # Get the list of columns to drop
    columns_to_drop = missing_above_threshold['Column'].tolist()

    # Remove the columns to keep from the list of columns to drop
    columns_to_drop = [col for col in columns_to_drop if col not in columns_to_keep]

    # Drop the columns from the dataset
    dataset = dataset.drop(columns=columns_to_drop)

    return dataset

# Load the dataset
crashes

# Specify the columns to keep
columns_to_keep = ['longitude', 'Latitude', 'location']

missing_data = calculate_missing_values(crashes)

# Print the missing_data dataframe to inspect the missing values and percentages
print(missing_data)

# Check the missing percentage specifically for the 'INJURIES_INCAPACITATING' column
missing_percentage_incapacitating = missing_data.loc[missing_data['Column'] == 'INJURIES_INCAPACITATING', 'Missing Percentage'].values[0]
print(f"Missing percentage for 'INJURIES_INCAPACITATING': {missing_percentage_incapacitating}%")

# Extract the columns with missing values exceeding 95% while retaining the specified columns
modified_data = extract_missing_values(crashes, 60, columns_to_keep)

modified_data

                           Column  Missing Values  Missing Percentage
0                 CRASH_RECORD_ID               0            0.000000
1                           RD_NO            4627            0.639565
2                CRASH_DATE_EST_I          668767           92.440080
3                      CRASH_DATE               0            0.000000
4              POSTED_SPEED_LIMIT               0            0.000000
5          TRAFFIC_CONTROL_DEVICE               0            0.000000
6                DEVICE_CONDITION               0            0.000000
7               WEATHER_CONDITION               0            0.000000
8              LIGHTING_CONDITION               0            0.000000
9                FIRST_CRASH_TYPE               0            0.000000
10                TRAFFICWAY_TYPE               0            0.000000
11                       LANE_CNT          524458           72.493020
12                      ALIGNMENT               0            0.000000
13           ROADWAY

,CRASH_RECORD_ID,RD_NO,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,JC199149,03/25/2019 02:43:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,ONE-WAY,...,0.0,1.0,2.0,0.0,14,2,3,41.884547,-87.641201,POINT (-87.64120093714 41.884547224337)
1,792b539deaaad65ee5b4a9691d927a34d298eb33d42af0...,JB422857,09/05/2018 08:40:00 AM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,NOT DIVIDED,...,0.0,0.0,2.0,0.0,8,4,9,41.968562,-87.740659,POINT (-87.740659314632 41.968562453871)
2,0115ade9a755e835255508463f7e9c4a9a0b47e9304238...,JF318029,07/15/2022 12:45:00 AM,30,UNKNOWN,UNKNOWN,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,NOT DIVIDED,...,0.0,0.0,2.0,0.0,0,6,7,41.886336,-87.716203,POINT (-87.716203130599 41.886336409761)
3,017040c61958d2fa977c956b2bd2d6759ef7754496dc96...,JF324552,07/15/2022 06:50:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,NOT DIVIDED,...,0.0,0.0,2.0,0.0,18,6,7,41.925111,-87.667997,POINT (-87.667997321599 41.925110815832)
4,78eee027ec3dcc85d36c9e3fdae4729dcc56440105d65b...,JB291672,06/03/2018 05:00:00 PM,30,NO CONTROLS,NO CONTROLS,CLEAR,UNKNOWN,PARKED MOTOR VEHICLE,ONE-WAY,...,0.0,0.0,1.0,0.0,17,1,6,41.910758,-87.731389,POINT (-87.731388754145 41.910757551599)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723455,722625096bc7b56c15fee9e09f4f1901fc5e25b8aacf2b...,JG251187,04/27/2023 05:47:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,OTHER,...,0.0,0.0,5.0,0.0,17,5,4,41.822237,-87.606996,POINT (-87.606995789103 41.822236685692)
723456,2d47afb7c4f2f4d305fddccb682ce850452476cc7f910d...,NaN,05/13/2023 03:54:00 PM,35,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,UNKNOWN,UNKNOWN,TURNING,NOT DIVIDED,...,0.0,0.0,2.0,0.0,15,7,5,41.852761,-87.625645,POINT (-87.625645007251 41.85276094394)
723457,4b1e7cbdb519f7f83fa3e3eae095ff54aebb73f125cf06...,NaN,05/11/2023 05:45:00 PM,30,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,UNKNOWN,UNKNOWN,ANGLE,NOT DIVIDED,...,0.0,0.0,4.0,0.0,17,5,5,41.891311,-87.646244,POINT (-87.646244110744 41.891311456709)
723458,5d3f02a062e775ef527557f75c4ded6900d21ad7108a3a...,NaN,05/13/2023 03:14:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,...,2.0,0.0,2.0,0.0,3,7,5,41.809485,-87.606711,POINT (-87.606710818433 41.809485427538)


In [86]:
# Identifying the columns and the data types in the merged datasets
def describe_columns(mdified_data):
    # Columns and their data types
    columns = modified_data.columns
    data_types = modified_data.dtypes
    
    # Create a DataFrame to store the information
    column_info = pd.DataFrame({'Column Name': columns, 'Data Type': data_types})
    
    # Print the column information
    print(column_info)

describe_columns(modified_data)

                                                 Column Name Data Type
CRASH_RECORD_ID                              CRASH_RECORD_ID    object
RD_NO                                                  RD_NO    object
CRASH_DATE                                        CRASH_DATE    object
POSTED_SPEED_LIMIT                        POSTED_SPEED_LIMIT     int64
TRAFFIC_CONTROL_DEVICE                TRAFFIC_CONTROL_DEVICE    object
DEVICE_CONDITION                            DEVICE_CONDITION    object
WEATHER_CONDITION                          WEATHER_CONDITION    object
LIGHTING_CONDITION                        LIGHTING_CONDITION    object
FIRST_CRASH_TYPE                            FIRST_CRASH_TYPE    object
TRAFFICWAY_TYPE                              TRAFFICWAY_TYPE    object
ALIGNMENT                                          ALIGNMENT    object
ROADWAY_SURFACE_COND                    ROADWAY_SURFACE_COND    object
ROAD_DEFECT                                      ROAD_DEFECT    object
REPORT

In [87]:
modified_data.drop(['INJURIES_INCAPACITATING'], axis=1, inplace=True)

In [88]:
modified_data.drop(['INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT','INJURIES_NO_INDICATION','INJURIES_UNKNOWN', 'INJURIES_TOTAL','INJURIES_FATAL'],axis =1, inplace=True)

In [89]:
modified_data

,CRASH_RECORD_ID,RD_NO,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,...,STREET_NAME,BEAT_OF_OCCURRENCE,NUM_UNITS,MOST_SEVERE_INJURY,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,JC199149,03/25/2019 02:43:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,ONE-WAY,...,RANDOLPH ST,121.0,2,"REPORTED, NOT EVIDENT",14,2,3,41.884547,-87.641201,POINT (-87.64120093714 41.884547224337)
1,792b539deaaad65ee5b4a9691d927a34d298eb33d42af0...,JB422857,09/05/2018 08:40:00 AM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,NOT DIVIDED,...,ELSTON AVE,1712.0,2,NO INDICATION OF INJURY,8,4,9,41.968562,-87.740659,POINT (-87.740659314632 41.968562453871)
2,0115ade9a755e835255508463f7e9c4a9a0b47e9304238...,JF318029,07/15/2022 12:45:00 AM,30,UNKNOWN,UNKNOWN,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,NOT DIVIDED,...,CENTRAL PARK AVE,1123.0,2,NO INDICATION OF INJURY,0,6,7,41.886336,-87.716203,POINT (-87.716203130599 41.886336409761)
3,017040c61958d2fa977c956b2bd2d6759ef7754496dc96...,JF324552,07/15/2022 06:50:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,NOT DIVIDED,...,ASHLAND AVE,1811.0,2,NO INDICATION OF INJURY,18,6,7,41.925111,-87.667997,POINT (-87.667997321599 41.925110815832)
4,78eee027ec3dcc85d36c9e3fdae4729dcc56440105d65b...,JB291672,06/03/2018 05:00:00 PM,30,NO CONTROLS,NO CONTROLS,CLEAR,UNKNOWN,PARKED MOTOR VEHICLE,ONE-WAY,...,KEELER AVE,2534.0,2,NO INDICATION OF INJURY,17,1,6,41.910758,-87.731389,POINT (-87.731388754145 41.910757551599)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723455,722625096bc7b56c15fee9e09f4f1901fc5e25b8aacf2b...,JG251187,04/27/2023 05:47:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,OTHER,...,COTTAGE GROVE AVE,214.0,2,NO INDICATION OF INJURY,17,5,4,41.822237,-87.606996,POINT (-87.606995789103 41.822236685692)
723456,2d47afb7c4f2f4d305fddccb682ce850452476cc7f910d...,NaN,05/13/2023 03:54:00 PM,35,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,UNKNOWN,UNKNOWN,TURNING,NOT DIVIDED,...,WABASH AVE,131.0,2,NO INDICATION OF INJURY,15,7,5,41.852761,-87.625645,POINT (-87.625645007251 41.85276094394)
723457,4b1e7cbdb519f7f83fa3e3eae095ff54aebb73f125cf06...,NaN,05/11/2023 05:45:00 PM,30,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,UNKNOWN,UNKNOWN,ANGLE,NOT DIVIDED,...,GRAND AVE,1214.0,2,NO INDICATION OF INJURY,17,5,5,41.891311,-87.646244,POINT (-87.646244110744 41.891311456709)
723458,5d3f02a062e775ef527557f75c4ded6900d21ad7108a3a...,NaN,05/13/2023 03:14:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,...,COTTAGE GROVE AVE,223.0,2,NONINCAPACITATING INJURY,3,7,5,41.809485,-87.606711,POINT (-87.606710818433 41.809485427538)


In [90]:
modified_data.isnull().sum()

CRASH_RECORD_ID                0
RD_NO                       4627
CRASH_DATE                     0
POSTED_SPEED_LIMIT             0
TRAFFIC_CONTROL_DEVICE         0
DEVICE_CONDITION               0
WEATHER_CONDITION              0
LIGHTING_CONDITION             0
FIRST_CRASH_TYPE               0
TRAFFICWAY_TYPE                0
ALIGNMENT                      0
ROADWAY_SURFACE_COND           0
ROAD_DEFECT                    0
REPORT_TYPE                20320
CRASH_TYPE                     0
DAMAGE                         0
DATE_POLICE_NOTIFIED           0
PRIM_CONTRIBUTORY_CAUSE        0
SEC_CONTRIBUTORY_CAUSE         0
STREET_NO                      0
STREET_DIRECTION               4
STREET_NAME                    1
BEAT_OF_OCCURRENCE             5
NUM_UNITS                      0
MOST_SEVERE_INJURY          1576
CRASH_HOUR                     0
CRASH_DAY_OF_WEEK              0
CRASH_MONTH                    0
LATITUDE                    4682
LONGITUDE                   4682
LOCATION  

In [91]:
modified_data['datetime'] = pd.to_datetime(modified_data['CRASH_DATE'])

In [92]:
modified_data['CRASH_MONTH'] = modified_data['datetime'].dt.month
modified_data['CRASH_DAY_OF_WEEK'] = modified_data['datetime'].dt.day
modified_data['CRASH_HOUR'] = modified_data['datetime'].dt.hour

In [93]:
modified_data.drop(['LOCATION', 'datetime', 'CRASH_DATE', 'CRASH_RECORD_ID','LATITUDE', 'LONGITUDE'], axis=1, inplace=True)

### One hot Encoding for first_crash_type

In [94]:
# To know which columns to perofrm one hot encoding , we need to identify categorical columns

categorical_columns = []
for column in modified_data.columns:
    if modified_data[column].nunique() > 2 and modified_data[column].dtype == 'object':
        categorical_columns.append(column)

categorical_columns


['RD_NO',
 'TRAFFIC_CONTROL_DEVICE',
 'DEVICE_CONDITION',
 'WEATHER_CONDITION',
 'LIGHTING_CONDITION',
 'FIRST_CRASH_TYPE',
 'TRAFFICWAY_TYPE',
 'ALIGNMENT',
 'ROADWAY_SURFACE_COND',
 'ROAD_DEFECT',
 'REPORT_TYPE',
 'DAMAGE',
 'DATE_POLICE_NOTIFIED',
 'PRIM_CONTRIBUTORY_CAUSE',
 'SEC_CONTRIBUTORY_CAUSE',
 'STREET_DIRECTION',
 'STREET_NAME',
 'MOST_SEVERE_INJURY']

In [107]:
def get_categorical_columns(dataframe):
    categorical_columns = dataframe.select_dtypes(include=['object', 'category']).columns.tolist()
    return categorical_columns

# Example usage
categorical_cols = get_categorical_columns(modified_data)
categorical_cols


['RD_NO',
 'TRAFFIC_CONTROL_DEVICE',
 'DEVICE_CONDITION',
 'WEATHER_CONDITION',
 'LIGHTING_CONDITION',
 'FIRST_CRASH_TYPE',
 'TRAFFICWAY_TYPE',
 'ALIGNMENT',
 'ROADWAY_SURFACE_COND',
 'ROAD_DEFECT',
 'REPORT_TYPE',
 'CRASH_TYPE',
 'DAMAGE',
 'DATE_POLICE_NOTIFIED',
 'SEC_CONTRIBUTORY_CAUSE',
 'STREET_DIRECTION',
 'STREET_NAME',
 'MOST_SEVERE_INJURY',
 'time_bins',
 'post_speed',
 'Contributory_Cause_Bins']

### Inspecting the values in each column 



In [95]:
for col in modified_data.columns:
    try:
        print(col, modified_data[col].value_counts(dropna=False)[:10])
    except:
        print(col, modified_data[col].value_counts())
        # If there aren't 5+ unique values for a column the first print statement
        
        # will throw an error for an invalid idx slice
    print('\n') # Break up the output between columns

RD_NO NaN         4627
JB486428       1
JC188143       1
JG137651       1
JD410659       1
JC134937       1
JA206508       1
JC248491       1
JC331966       1
JC155968       1
Name: RD_NO, dtype: int64


POSTED_SPEED_LIMIT 30    531922
35     48545
25     45547
20     30169
15     25586
10     16682
0       7294
40      7006
45      4713
5       4423
Name: POSTED_SPEED_LIMIT, dtype: int64


TRAFFIC_CONTROL_DEVICE NO CONTROLS               413817
TRAFFIC SIGNAL            200394
STOP SIGN/FLASHER          71768
UNKNOWN                    26756
OTHER                       4804
LANE USE MARKING            1226
YIELD                       1030
OTHER REG. SIGN              754
OTHER WARNING SIGN           609
RAILROAD CROSSING GATE       469
Name: TRAFFIC_CONTROL_DEVICE, dtype: int64


DEVICE_CONDITION NO CONTROLS                 418751
FUNCTIONING PROPERLY        248744
UNKNOWN                      44371
OTHER                         5565
FUNCTIONING IMPROPERLY        3481
NOT FUNCTIONING 

## Creating bins for crash_hour

In [96]:
# observing values of crash_hour
modified_data.CRASH_HOUR.value_counts()

15    55640
16    55210
17    53895
14    48609
18    44533
13    44176
12    42690
8     38130
11    36874
9     33271
10    32892
19    32798
7     30518
20    26501
21    23655
22    21659
23    18787
6     15744
0     15565
1     13263
2     11451
5      9941
3      9322
4      8336
Name: CRASH_HOUR, dtype: int64

In [97]:
# creating bins for times 
# 0-6 = Late Night/Early Morning 
# 6-12 = Morning 
# 12-18 = Afternoon/Rush Hour
# 18-23 = Late Evening 
modified_data['time_bins'] = pd.cut(x=modified_data['CRASH_HOUR'], bins = [0,6,12,18,23], 
                         labels = ['Late Night/Early Morning', 
                        'Morning', 'Afternoon/Rush Hour','Late Evening'])
modified_data.head()

,RD_NO,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,ALIGNMENT,ROADWAY_SURFACE_COND,...,STREET_NO,STREET_DIRECTION,STREET_NAME,BEAT_OF_OCCURRENCE,NUM_UNITS,MOST_SEVERE_INJURY,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,time_bins
0,JC199149,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,ONE-WAY,STRAIGHT AND LEVEL,DRY,...,536,W,RANDOLPH ST,121.0,2,"REPORTED, NOT EVIDENT",14,25,3,Afternoon/Rush Hour
1,JB422857,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,...,4821,N,ELSTON AVE,1712.0,2,NO INDICATION OF INJURY,8,5,9,Morning
2,JF318029,30,UNKNOWN,UNKNOWN,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,...,300,N,CENTRAL PARK AVE,1123.0,2,NO INDICATION OF INJURY,0,15,7,NaN
3,JF324552,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,...,2359,N,ASHLAND AVE,1811.0,2,NO INDICATION OF INJURY,18,15,7,Afternoon/Rush Hour
4,JB291672,30,NO CONTROLS,NO CONTROLS,CLEAR,UNKNOWN,PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,DRY,...,1630,N,KEELER AVE,2534.0,2,NO INDICATION OF INJURY,17,3,6,Afternoon/Rush Hour


### Create post_speed_limit bins

In [98]:
modified_data.POSTED_SPEED_LIMIT.value_counts()

30    531922
35     48545
25     45547
20     30169
15     25586
10     16682
0       7294
40      7006
45      4713
5       4423
55       716
50       184
3        165
9         95
39        74
99        66
60        44
1         39
24        36
2         26
32        17
65        17
34        14
33        13
11        11
6          7
26         6
36         5
7          5
70         4
22         3
14         3
12         3
29         3
8          2
4          2
31         2
23         2
38         2
18         2
44         1
63         1
62         1
16         1
49         1
Name: POSTED_SPEED_LIMIT, dtype: int64

In [99]:
#we create the bins and label for better preview
# creating bins and label, previewing data 
modified_data['post_speed'] = pd.cut(x=modified_data['POSTED_SPEED_LIMIT'], bins = [0,15,25,40,75], labels = ['0-15', '16-25', '26-40', '41+'])
modified_data.head()

,RD_NO,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,ALIGNMENT,ROADWAY_SURFACE_COND,...,STREET_DIRECTION,STREET_NAME,BEAT_OF_OCCURRENCE,NUM_UNITS,MOST_SEVERE_INJURY,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,time_bins,post_speed
0,JC199149,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,ONE-WAY,STRAIGHT AND LEVEL,DRY,...,W,RANDOLPH ST,121.0,2,"REPORTED, NOT EVIDENT",14,25,3,Afternoon/Rush Hour,26-40
1,JB422857,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,...,N,ELSTON AVE,1712.0,2,NO INDICATION OF INJURY,8,5,9,Morning,26-40
2,JF318029,30,UNKNOWN,UNKNOWN,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,...,N,CENTRAL PARK AVE,1123.0,2,NO INDICATION OF INJURY,0,15,7,NaN,26-40
3,JF324552,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,...,N,ASHLAND AVE,1811.0,2,NO INDICATION OF INJURY,18,15,7,Afternoon/Rush Hour,26-40
4,JB291672,30,NO CONTROLS,NO CONTROLS,CLEAR,UNKNOWN,PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,DRY,...,N,KEELER AVE,2534.0,2,NO INDICATION OF INJURY,17,3,6,Afternoon/Rush Hour,26-40


In [100]:
contributory_causes = modified_data['PRIM_CONTRIBUTORY_CAUSE'].unique()
print(contributory_causes)


['IMPROPER TURNING/NO SIGNAL'
 'VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ETC.)'
 'UNABLE TO DETERMINE' 'NOT APPLICABLE' 'IMPROPER LANE USAGE' 'WEATHER'
 'DISREGARDING TRAFFIC SIGNALS' 'FAILING TO YIELD RIGHT-OF-WAY'
 'FOLLOWING TOO CLOSELY'
 'UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ARREST IS EFFECTED)'
 'TURNING RIGHT ON RED' 'IMPROPER BACKING'
 'CELL PHONE USE OTHER THAN TEXTING'
 'FAILING TO REDUCE SPEED TO AVOID CRASH' 'IMPROPER OVERTAKING/PASSING'
 'DRIVING SKILLS/KNOWLEDGE/EXPERIENCE' 'EQUIPMENT - VEHICLE CONDITION'
 'DISTRACTION - FROM INSIDE VEHICLE' 'DISTRACTION - FROM OUTSIDE VEHICLE'
 'OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER'
 'DISREGARDING ROAD MARKINGS' 'DISREGARDING STOP SIGN'
 'DISREGARDING OTHER TRAFFIC SIGNS' 'EXCEEDING AUTHORIZED SPEED LIMIT'
 'PHYSICAL CONDITION OF DRIVER' 'RELATED TO BUS STOP'
 'DRIVING ON WRONG SIDE/WRONG WAY' 'ROAD CONSTRUCTION/MAINTENANCE'
 'ROAD ENGINEERING/SURFACE/MARKING DEFECTS' 'ANIMAL'
 '

In [102]:
unique_values = modified_data['PRIM_CONTRIBUTORY_CAUSE'].unique()

In [103]:
# Define the mapping dictionary
mapping_dict = {
    'Driver Inattention/Distraction': 'Inattention/Distraction',
    'Failure to Yield Right-of-Way': 'Failure to Yield',
    'Other': 'Other',
    # Add more mappings as needed
}

# Apply the mapping
modified_data['Contributory_Cause_Bins'] = modified_data['PRIM_CONTRIBUTORY_CAUSE'].map(mapping_dict)


In [104]:
modified_data['PRIM_CONTRIBUTORY_CAUSE'] = pd.to_numeric(modified_data['PRIM_CONTRIBUTORY_CAUSE'], errors='coerce')
bins = [0, 3, 6, 9, 12]
labels = ['Low', 'Medium', 'High', 'Very High']
modified_data['Contributory_Cause_Bins'] = pd.cut(modified_data['PRIM_CONTRIBUTORY_CAUSE'], bins=bins, labels=labels)


In [105]:
# Define the bins
bins = [0, 3, 6, 9, 12]  # Example bins, adjust them according to your needs

# Create the bin labels
labels = ['Low', 'Medium', 'High', 'Very High']  # Example labels, adjust them according to your needs

# Apply binning to the column
modified_data['Contributory_Cause_Bins'] = pd.cut(modified_data['PRIM_CONTRIBUTORY_CAUSE'], bins=bins, labels=labels)

modified_data.head()

,RD_NO,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,ALIGNMENT,ROADWAY_SURFACE_COND,...,STREET_NAME,BEAT_OF_OCCURRENCE,NUM_UNITS,MOST_SEVERE_INJURY,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,time_bins,post_speed,Contributory_Cause_Bins
0,JC199149,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,ONE-WAY,STRAIGHT AND LEVEL,DRY,...,RANDOLPH ST,121.0,2,"REPORTED, NOT EVIDENT",14,25,3,Afternoon/Rush Hour,26-40,NaN
1,JB422857,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,...,ELSTON AVE,1712.0,2,NO INDICATION OF INJURY,8,5,9,Morning,26-40,NaN
2,JF318029,30,UNKNOWN,UNKNOWN,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,...,CENTRAL PARK AVE,1123.0,2,NO INDICATION OF INJURY,0,15,7,NaN,26-40,NaN
3,JF324552,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,...,ASHLAND AVE,1811.0,2,NO INDICATION OF INJURY,18,15,7,Afternoon/Rush Hour,26-40,NaN
4,JB291672,30,NO CONTROLS,NO CONTROLS,CLEAR,UNKNOWN,PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,DRY,...,KEELER AVE,2534.0,2,NO INDICATION OF INJURY,17,3,6,Afternoon/Rush Hour,26-40,NaN
